In [5]:
#kaggle那个数据集不好用😠，随便又找了一个
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = pd.read_csv('/Users/branch/Desktop/boston_housing_data.csv')
print(data.shape)

# 处理一下这个美丽的数据🤩
data = data.interpolate(method = 'linear').fillna(data.mean())
#删除一下波动较大的第二列
data = data.drop(['ZN'], axis=1)


x_data = data.iloc[:, :12].values
y_data = data.iloc[:, 12].values
print(x_data.shape)
print(y_data.shape)

# 标准化
scaler = StandardScaler()
scaler.fit(x_data)
x_data = scaler.transform(x_data)


X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1, random_state=56)
train_x = torch.from_numpy(X_train.astype(np.float32))
train_y = torch.from_numpy(y_train.astype(np.float32)).view(-1, 1)  
test_x = torch.from_numpy(X_test.astype(np.float32))
test_y = torch.from_numpy(y_test.astype(np.float32)).view(-1, 1)  

train_data = TensorDataset(train_x, train_y)
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)

# 定义一下模型😄
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(12, 128),  
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 1)
        )
    
    def forward(self, x):
        return self.fc(x)

model = Model().to(device)

#损失函数和优化器
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-4)

# training（0_0)
num_epochs = 100
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        # 前向
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        # 反向优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# test并评估
model.eval()
with torch.no_grad():
    test_x, test_y = test_x.to(device), test_y.to(device)
    test_outputs = model(test_x)
    test_loss = criterion(test_outputs, test_y)
    print(f'Test Loss: {test_loss.item():.4f}')


(506, 14)
(506, 12)
(506,)
Epoch [1/100], Loss: 324.9702
Epoch [2/100], Loss: 339.4274
Epoch [3/100], Loss: 413.8306
Epoch [4/100], Loss: 235.2878
Epoch [5/100], Loss: 321.8720
Epoch [6/100], Loss: 69.2809
Epoch [7/100], Loss: 80.9807
Epoch [8/100], Loss: 296.1701
Epoch [9/100], Loss: 25.4659
Epoch [10/100], Loss: 32.2750
Epoch [11/100], Loss: 8.7093
Epoch [12/100], Loss: 42.3990
Epoch [13/100], Loss: 32.9533
Epoch [14/100], Loss: 38.9646
Epoch [15/100], Loss: 15.2752
Epoch [16/100], Loss: 33.0249
Epoch [17/100], Loss: 32.2351
Epoch [18/100], Loss: 32.3216
Epoch [19/100], Loss: 16.5615
Epoch [20/100], Loss: 6.0306
Epoch [21/100], Loss: 16.3166
Epoch [22/100], Loss: 7.2475
Epoch [23/100], Loss: 30.0824
Epoch [24/100], Loss: 24.5348
Epoch [25/100], Loss: 12.6873
Epoch [26/100], Loss: 10.7320
Epoch [27/100], Loss: 24.2792
Epoch [28/100], Loss: 8.9975
Epoch [29/100], Loss: 31.0543
Epoch [30/100], Loss: 12.0931
Epoch [31/100], Loss: 4.5321
Epoch [32/100], Loss: 15.9143
Epoch [33/100], Loss: